In [1]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [12]:
# load ascii text and covert to lowercase
filename = "data/products.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  527
Total Vocab:  36


In [13]:
# Load Larger LSTM network and generate text


# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

Total Patterns:  427


In [15]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=10, batch_size=64, callbacks=callbacks_list)

Epoch 1/10
384/427 [=========================>....] - ETA: 2s - loss: 3.4682
Epoch 00001: loss improved from inf to 3.37348, saving model to weights-improvement-01-3.3735-bigger.hdf5
427/427 [==============================] - 21s 49ms/step - loss: 3.3735
Epoch 2/10
384/427 [=========================>....] - ETA: 1s - loss: 3.1644
Epoch 00002: loss improved from 3.37348 to 3.16337, saving model to weights-improvement-02-3.1634-bigger.hdf5
427/427 [==============================] - 18s 43ms/step - loss: 3.1634
Epoch 3/10
384/427 [=========================>....] - ETA: 1s - loss: 3.0727
Epoch 00003: loss improved from 3.16337 to 3.05539, saving model to weights-improvement-03-3.0554-bigger.hdf5
427/427 [==============================] - 20s 46ms/step - loss: 3.0554
Epoch 4/10
384/427 [=========================>....] - ETA: 1s - loss: 3.0673
Epoch 00004: loss did not improve
427/427 [==============================] - 18s 42ms/step - loss: 3.0628
Epoch 5/10
384/427 [========================

In [22]:

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
res = []
for i in range(10):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
#     sys.stdout.write(result)
    res.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print(res)
print ("\nDone.")

Seed:
" rce plus
 chaga
 caveman true paleo formula
 colloidal silver
 silver bullet
 sun potion
 wake up am "
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']

Done.
